# Boosting Algerian Agricultural Company Profits

<font color="#fcd695"> The goal from this project in to create a model that can predict the improvement of salles of agricultural products for a specific agricultural company in other words, we want to dectect how to boost the profit of Algerian Agricultural Company </strong></font>


## 1 Steps Of The Project:

### 1.1 Understand the Business/Research Context:

<font color="#ffe9c4">Gain a deep understanding of the business or research context. Understand the domain, business goals, and specific challenges.</font>

### 1.2 Data presentation:

<font color="#ffe9c4">
The dataset is csv file that contains a comprehensive dataset capturing various transactions within an agriculture company.The dataset has this features: 


 </font>

 | Column                  | Description                                              |
|-------------------------|----------------------------------------------------------|
| CleEffet                | Identifier for transactions.                             |
| Date                    | Date of the transaction.                                  |
| TypeEffet               | Type of transaction.                                      |
| FamilleArticle          | Family/category of the transaction product.              |
| CodeProduit             | Product code.                                            |
| ReferenceProduit        | Reference code for the product.                           |
| NLot                    | Lot number.                                              |
| Produit_Designation     | Original designation of the product of a transaction detail.|
| DetailEffet_Designation | Designation (sometimes different from Produit_Designation) of a transaction detail.|
| Quantite                | Quantity of the product.                                  |
| PrixAchat               | Purchase price.                                         |
| PrixUnitaireTTC         | Unit price including taxes.                              |
| DetailEffet_Marge       | Margin for the transaction detail.                        |
| Effet_Marge             | Margin for the transaction.                               |
| MontantHT               | Transaction amount excluding taxes.                      |
| Payement                | Payment amount.                                          |



In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv("./Project_datamining_final_data_03_12_an1.csv")
print(data.info())
print(data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261001 entries, 0 to 261000
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CleEffet                 261001 non-null  int64  
 1   Date                     261001 non-null  object 
 2   TypeEffet                261001 non-null  object 
 3   FamilleArticle           261001 non-null  object 
 4   Code Produit             160237 non-null  object 
 5   Reference Produit        135717 non-null  object 
 6   NLot                     228637 non-null  object 
 7   Produit_Designation      261001 non-null  object 
 8   DetailEffet_Designation  261001 non-null  object 
 9   Quantite                 261001 non-null  float64
 10  PrixAchat                261001 non-null  object 
 11  PrixUnitaireTTC          261001 non-null  object 
 12  DetailEffet_Marge        261001 non-null  object 
 13  Effet_Marge              261001 non-null  object 
 14  Mont

In [5]:
data.head()

,CleEffet,Date,TypeEffet,FamilleArticle,Code Produit,Reference Produit,NLot,Produit_Designation,DetailEffet_Designation,Quantite,PrixAchat,PrixUnitaireTTC,DetailEffet_Marge,Effet_Marge,MontantHT,Payement
0,1,8/14/2012,Bon d'entrée,Produit Insecticides,0305-004,0305,CHA110501,Produit_Anon_0,Detail_Anon_0,65.0,"1,650.00","9,999.00",0.00,"216,310,866.75","216,310,866.75",0.00
1,1,8/14/2012,Bon d'entrée,Matériel bovin,11250,0305,10.274.1.09,Produit_Anon_1,Detail_Anon_1,120.0,550.00,"9,999.00",0.00,"216,310,866.75","216,310,866.75",0.00
2,1,8/14/2012,Bon d'entrée,Produit Insecticides,0305-039,0305,L110703,Produit_Anon_2,Detail_Anon_2,137.0,"1,034.00","9,999.00",0.00,"216,310,866.75","216,310,866.75",0.00
3,1,8/14/2012,Bon d'entrée,Produit Insecticides,2F18272093,2F18272093,2F18272093,Produit_Anon_3,Detail_Anon_3,2000.0,"2,800.00","9,999.00",0.00,"216,310,866.75","216,310,866.75",0.00
4,1,8/14/2012,Bon d'entrée,Produit Insecticides,0305-054,0305,20101102,Produit_Anon_4,Detail_Anon_4,3500.0,"1,550.00","9,999.00",0.00,"216,310,866.75","216,310,866.75",0.00


In [6]:
# Data Types Assingment
data['CleEffet'] = data['CleEffet'].astype(str)
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%Y')
data['TypeEffet'] = data['TypeEffet'].astype(str)
data['FamilleArticle'] = data['FamilleArticle'].astype(str)
data['Code Produit'] = data['Code Produit'].astype(str)
data['Reference Produit'] = data['Reference Produit'].astype(str)
data['NLot'] = data['NLot'].astype(str)
data['Produit_Designation'] = data['Produit_Designation'].astype(str)
data['DetailEffet_Designation'] = data['DetailEffet_Designation'].astype(str)
data['Code Produit'] = data['Code Produit'].astype(str)

data['PrixAchat'] = data['PrixAchat'].str.replace(',', '').astype('float64')
data['PrixUnitaireTTC'] = data['PrixUnitaireTTC'].str.replace(',', '').astype('float64')
data['DetailEffet_Marge'] = data['DetailEffet_Marge'].str.replace(',', '').astype('float64')
data['Effet_Marge'] = data['Effet_Marge'].str.replace(',', '').astype('float64')
data['MontantHT'] = data['MontantHT'].str.replace(',', '').astype('float64')
data['Payement'] = data['Payement'].str.replace(',', '').astype('float64')

In [7]:
data['TypeEffet'].value_counts()

TypeEffet
Bon de livraison    223143
Bon de réception     18098
Bon de sortie        11451
Bon d'entrée          8309
Name: count, dtype: int64

In [8]:
data2013 = data[data['Date'].dt.year == 2013]
data2013.head()

,CleEffet,Date,TypeEffet,FamilleArticle,Code Produit,Reference Produit,NLot,Produit_Designation,DetailEffet_Designation,Quantite,PrixAchat,PrixUnitaireTTC,DetailEffet_Marge,Effet_Marge,MontantHT,Payement
7407,4721,2013-01-01,Bon de livraison,Matériel apiculture,6116,6000,1005125706,Produit_Anon_414,Detail_Anon_429,2.0,2395.0,2750.0,5398.00,5398.00,5500.0,5400.0
7408,4722,2013-01-01,Bon de livraison,Semences Avoine,0124-002,0124,no lot,Produit_Anon_1482,Detail_Anon_2122,33.0,35.0,60.0,226.55,247.49,2225.0,2220.0
7409,4722,2013-01-01,Bon de livraison,SEMENCES SEC,0132-001,0132,no lot,Produit_Anon_1820,Detail_Anon_1858,7.0,1900.0,35.0,20.94,247.49,2225.0,2220.0
7410,4723,2013-01-01,Bon de livraison,Produit Insecticides,0305-071,0305,1002,Produit_Anon_1813,Detail_Anon_1841,1.0,1200.0,1100.0,300.00,300.00,1100.0,1100.0
7411,4724,2013-01-01,Bon de livraison,Semences piment,0108-005,0108,2292,Produit_Anon_2058,Detail_Anon_2249,40.0,670.0,650.0,2000.00,2000.00,26000.0,26000.0


In [9]:
data['TypeEffet'].value_counts()
data.query('TypeEffet == "Bon de livraison" and CleEffet > "546"')
data.query('(PrixUnitaireTTC - PrixAchat) * Quantite != DetailEffet_Marge and TypeEffet == "Bon de livraison"' )




,CleEffet,Date,TypeEffet,FamilleArticle,Code Produit,Reference Produit,NLot,Produit_Designation,DetailEffet_Designation,Quantite,PrixAchat,PrixUnitaireTTC,DetailEffet_Marge,Effet_Marge,MontantHT,Payement
623,340,2012-09-03,Bon de livraison,Detail,DET,DET,no lot,Produit_Anon_259,Detail_Anon_268,1.0,126.0,300.0,299.00,299.00,300.0,0.0
624,341,2012-09-03,Bon de livraison,Detail,DET,DET,no lot,Produit_Anon_259,Detail_Anon_268,150.0,126.0,1.0,0.00,0.00,150.0,0.0
2166,540,2012-09-08,Bon de livraison,Matériels Tronceneuses,30036,30000,no lot,Produit_Anon_465,Detail_Anon_480,1.0,180.0,250.0,249.00,249.00,250.0,250.0
2172,546,2012-09-08,Bon de livraison,Detail,DET,DET,no lot,Produit_Anon_259,Detail_Anon_268,6.0,126.0,250.0,1494.00,1494.00,1500.0,1500.0
2173,547,2012-09-08,Bon de livraison,Produit Insecticides,0305-045,0305,111008,Produit_Anon_27,Detail_Anon_27,1.0,1200.0,500.0,-9515.13,-18498.00,1550.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260989,68345,2022-07-27,Bon de livraison,Produit Engrais,00067,nan,1330328,Produit_Anon_6772,Detail_Anon_10819,1.0,9075.0,11000.0,1906.21,9758.58,101765.0,0.0
260990,68345,2022-07-27,Bon de livraison,Semences haricot,nan,nan,G2201129,Produit_Anon_2228,Detail_Anon_6197,3.0,11000.0,11500.0,1120.30,9758.58,101765.0,0.0
260991,68345,2022-07-27,Bon de livraison,Produit Insecticides,0305-039,0305,l211006,Produit_Anon_2,Detail_Anon_11982,1.0,2400.0,2900.0,514.24,9758.58,101765.0,0.0
260992,68345,2022-07-27,Bon de livraison,Courgette_Hybride,nan,nan,GS834-30-322,Produit_Anon_7114,Detail_Anon_11731,5.0,4300.0,4500.0,752.37,9758.58,101765.0,0.0


In [10]:
data2013Livraison = data2013[data2013['TypeEffet'] == 'Bon de livraison']


# Extract month and year from the 'Date' column
data2013Livraison['Month'] = data2013Livraison['Date'].dt.month

# Group by 'FamilleArticle' and 'Month' and sum the 'Quantite' for each group
grouped = data2013Livraison.groupby(['FamilleArticle', 'Month'])['DetailEffet_Marge'].sum().reset_index()

# Splitting categories into groups of 10 for plotting
categories = grouped['FamilleArticle'].unique()

for i in range(len(categories) // 10 + 1):
    plt.figure(figsize=(10, 6))
    
    for j in range(i * 10, min((i + 1) * 10, len(categories))):
        plt.title(f'Categories {i * 10 + 1} to {min((i + 1) * 10, len(categories))}')
        
        category = categories[j]
        category_data = grouped[grouped['FamilleArticle'] == category]
        plt.plot(category_data['Month'], category_data['DetailEffet_Marge'], label=category)
        
        plt.xlabel('Months')
        plt.ylabel('DetailEffet_Marge')
    
    plt.legend()
    plt.tight_layout()
    
    # Save each set of 10 categories as an individual image file
    plt.savefig(f'plot_categories_{i * 10 + 1}_to_{min((i + 1) * 10, len(categories))}.png')
    plt.close()  # Close the current figure to prevent overlapping plots

print("Plots saved as individual image files.")

C:\Users\pc\AppData\Local\Temp\ipykernel_3468\235756049.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2013Livraison['Month'] = data2013Livraison['Date'].dt.month


Plots saved as individual image files.


In [15]:
# Define a custom function to sort the DataFrame within each group
def sort_by_detail_marge(group):
    return group.sort_values(by='DetailEffet_Marge', ascending=False)

# Apply the custom function to each group within each month
sorted_grouped = grouped.groupby('Month', group_keys=False).apply(sort_by_detail_marge)

# Path to save PDF files
pdf_path = 'tables2013'

# Iterate over each month
for month, month_group in sorted_grouped.groupby('Month'):
    # Calculate the percentage column and round to two decimal places
    month_group['Percentage'] = (month_group['DetailEffet_Marge'] / month_group['DetailEffet_Marge'].sum() * 100).round(2)

    # Round 'DetailEffet_Marge' to two decimal places
    month_group['DetailEffet_Marge'] = month_group['DetailEffet_Marge'].round(2)

    # Create a table using matplotlib
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')

    # Include 'Percentage' column in the table
    table_data = month_group[['FamilleArticle', 'DetailEffet_Marge', 'Percentage']].values
    col_labels = ['FamilleArticle', 'DetailEffet_Marge', 'Percentage']
    ax.table(cellText=table_data,
             colLabels=col_labels,
             cellLoc='center', loc='center')

    # Save the table as a PDF file
    pdf_filename = f'{pdf_path}/table_{month}.pdf'
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    plt.close()

In [16]:
data2013Livraison = data[data['TypeEffet'] == 'Bon de livraison']


# Extract month and year from the 'Date' column
data2013Livraison['Month'] = data2013Livraison['Date'].dt.month

# Group by 'FamilleArticle' and 'Month' and sum the 'Quantite' for each group
grouped = data2013Livraison.groupby(['FamilleArticle', 'Month'])['DetailEffet_Marge'].sum().reset_index()


# Define a custom function to sort the DataFrame within each group
def sort_by_detail_marge(group):
    return group.sort_values(by='DetailEffet_Marge', ascending=False)

# Apply the custom function to each group within each month
sorted_grouped = grouped.groupby('Month', group_keys=False).apply(sort_by_detail_marge)

# Path to save PDF files
pdf_path = 'tables'

# Iterate over each month
for month, month_group in sorted_grouped.groupby('Month'):
    # Calculate the percentage column and round to two decimal places
    month_group['Percentage'] = (month_group['DetailEffet_Marge'] / month_group['DetailEffet_Marge'].sum() * 100).round(2)

    # Round 'DetailEffet_Marge' to two decimal places
    month_group['DetailEffet_Marge'] = month_group['DetailEffet_Marge'].round(2)

    # Create a table using matplotlib
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')

    # Include 'Percentage' column in the table
    table_data = month_group[['FamilleArticle', 'DetailEffet_Marge', 'Percentage']].values
    col_labels = ['FamilleArticle', 'DetailEffet_Marge', 'Percentage']
    ax.table(cellText=table_data,
             colLabels=col_labels,
             cellLoc='center', loc='center')

    # Save the table as a PDF file
    pdf_filename = f'{pdf_path}/table_{month}.pdf'
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    plt.close()


C:\Users\pc\AppData\Local\Temp\ipykernel_3468\2830572646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2013Livraison['Month'] = data2013Livraison['Date'].dt.month


In [21]:
data2013Livraison[data2013Livraison['FamilleArticle'] == 'Courgette_Hybride']
data2013Livraison.query('FamilleArticle == "Courgette_Hybride" and Effet_Marge < 0')

,CleEffet,Date,TypeEffet,FamilleArticle,Code Produit,Reference Produit,NLot,Produit_Designation,DetailEffet_Designation,Quantite,PrixAchat,PrixUnitaireTTC,DetailEffet_Marge,Effet_Marge,MontantHT,Payement,Month
46409,28007,2014-08-05,Bon de livraison,Courgette_Hybride,0204-013,0204,11585885,Produit_Anon_1470,Detail_Anon_5463,1.0,3200.00,3550.0,2.790900e+02,-2.604944e+04,133290.0,132504.99,8
46423,28007,2014-08-05,Bon de livraison,Courgette_Hybride,0204-012,0204,F25026120101T0,Produit_Anon_1469,Detail_Anon_3874,1.0,5800.00,6200.0,3.891600e+02,-2.604944e+04,133290.0,132504.99,8
54626,29942,2015-02-07,Bon de livraison,Courgette_Hybride,0104-011,0104,4 A 0351/5,Produit_Anon_157,Detail_Anon_157,1.0,2400.00,2900.0,-4.000000e+10,-3.999994e+10,501944.0,501409.00,2
56270,30194,2015-03-12,Bon de livraison,Courgette_Hybride,0104-007,0104,101726800,Produit_Anon_159,Detail_Anon_5971,2.0,6400.00,7000.0,1.122270e+03,-1.749042e+05,502490.0,499699.99,3
56836,30245,2015-03-19,Bon de livraison,Courgette_Hybride,0104-007,0104,101726800,Produit_Anon_159,Detail_Anon_5971,1.0,6400.00,7300.0,8.263000e+02,-8.395600e+02,594270.0,588270.00,3
56837,30245,2015-03-19,Bon de livraison,Courgette_Hybride,0104-007,0104,101726800,Produit_Anon_159,Detail_Anon_5971,1.0,6400.00,7100.0,6.283200e+02,-8.395600e+02,594270.0,588270.00,3
61230,30949,2015-05-05,Bon de livraison,Courgette_Hybride,0204-012,0204,F25026110101T1,Produit_Anon_1469,Detail_Anon_3874,1.0,5750.00,5062.5,-7.118300e+02,-2.847320e+03,20250.0,0.00,5
61231,30949,2015-05-05,Bon de livraison,Courgette_Hybride,0204-012,0204,F25026120101T0,Produit_Anon_1469,Detail_Anon_3874,3.0,5800.00,5062.5,-2.135490e+03,-2.847320e+03,20250.0,0.00,5
206596,58231,2020-08-17,Bon de livraison,Courgette_Hybride,0204-012,0204,NO5814180201T0,Produit_Anon_1469,Detail_Anon_9836,2.0,8849.99,9500.0,2.935000e+03,-2.065000e+03,89000.0,0.00,8
207908,58464,2020-09-16,Bon de livraison,Courgette_Hybride,0204-012,0204,NO5814180201T0,Produit_Anon_1469,Detail_Anon_9836,2.0,8849.99,9000.0,1.900200e+03,-7.115156e+04,124140.0,0.00,9


### 1.3 Data Cleaning and Preprocessing:

<font color="#ffe9c4">At this stages we should improve the quality of our data, and make it easy to use and work with by doing the following:</font>

#### 1.3.1 Data Cleanning:
Clean the raw data by handling missing values, outliers, and inconsistencies.



#### 1.3.2 Data Preprocessing:
Preprocess the data to transform it into a suitable format (type) for analysis since most of our data are of type object. This step include also normalization, standardization, and encoding categorical variables and also transforming the attributes names and content to english since they are written in french.

### 1.4 Exploratory Data Analysis (EDA):

<font color="#ffe9c4">Explore the data to gain insights and an initial understanding of its characteristics. This may involve the use of statistical summaries, visualizations, and other exploratory techniques.</font>

### 1.5 Feature Selection/Engineering:

<font color="#ffe9c4">Select relevant features (variables) for analysis and, if needed, create new features that may enhance the predictive power of the model.</font>

### 1.6 Data Splitting:

<font color="#ffe9c4">Divide the dataset into training, validation, and test sets. The training set is used to train the model, the validation set is used to tune hyperparameters, and the test set is used to evaluate the model's performance.</font>



### 1.7 Model Selection:

<font color="#ffe9c4">Choose the appropriate data mining algorithm(s) based on the nature of the problem and the characteristics of the data. Common algorithms include decision trees, clustering, regression, and neural networks.</font>


### 1.8 Model Training:

<font color="#ffe9c4">Train the selected model(s) on the training dataset. This involves using historical data to teach the model to recognize patterns and relationships.</font>


### 1.9 Model Evaluation:

<font color="#ffe9c4">Assess the performance of the model using the validation set. This may involve metrics such as accuracy, precision, recall, F1 score, or area under the receiver operating characteristic curve (AUC-ROC), depending on the nature of the problem (classification, regression, etc.).</font>